In [ ]:
library(tidyverse)
library(magrittr)
library(DT)

options(repr.matrix.max.rows=20, repr.matrix.max.cols=30)

In [ ]:
airport_codes <- data.table::fread('http://ourairports.com/data/airports')

In [ ]:
airport_codes

In [ ]:
airport_codes %>% select(ident, type, continent, iso_country)
#selection of identification number, type, continent and country columns

In [ ]:
airport_codes %>%
filter(type== "small_airport" & iso_country== "US" & elevation_ft > 3000)
# filtering small airports in the US with altitude higher than 3000 ft

In [ ]:
airport_codes %<>%
mutate(elevation_m = (elevation_ft * 0.305)) %>%
mutate(elevation_km = elevation_m / 1000)
# making a mutate operation to calculate elevation in meters and piping the meter values into kilometers

In [ ]:
airport_codes

In [ ]:
airport_codes %>%
group_by(type, iso_country) %>%
summarise(max_elevation = max(elevation_ft, na.rm = T))
# piping group by and summarise operations to find the maximum elevation of each airport type in each country
# code is returning minus inf if no combination is found

In [ ]:
#finds the min, max, and mean altitude of each type of airport in each country

airport_altitudes <- airport_codes %>%
group_by(type, iso_country) %>%
summarise(max_altitude_tc = max(elevation_ft, na.rm = T),
          min_altitude_tc = min(elevation_ft, na.rm = T),
          av_altitude_tc = mean(elevation_ft, na.rm = T))

airport_altitudes

In [ ]:
#making a pivot wide operation
airport_at_wide <- airport_altitudes %>% pivot_wider(id_cols = type, names_from = iso_country, values_from = av_altitude_tc)
airport_at_wide

In [ ]:
# making a pivot long operation
airport_at_long <- airport_at_wide %>% pivot_longer(cols = -"type", names_to = "iso_country", values_to = "av_altitude_tc", values_drop_na = T)
airport_at_long

In [ ]:
# making a right join, first split the table into 2 countries to have 2 different tables

airports_us <- airport_codes %>%
filter(iso_country == "US") %>% slice(1:5) # filtering airports in US

airports_fr <- airport_codes %>%
filter(iso_country== "FR") %>% slice(1:5) # filtering airports in France

airports_us
airports_fr

In [ ]:
# making a left join to find the airports in France that has the same type as the airports in the the US

airports_us %>% left_join(airports_fr, by = "type")